## 2-6difluoropyridine Adsorption Energy Example

## You have to visualize the structures after relaxation!!

In [40]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/2_6difluoropyridine_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [41]:
xyz_path = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ni_oct_2_2_6difluoropyridine_5_hydride_1_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

$$\Delta E_{ads} = (E_{complex+H}+E_{separate ligand}) - E_{complex} - E_H$$
### We need to add the energy of the separate ligand because in the hydride complex, we remove a ligand to add the hydrogen.

In [42]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "Fc1cccc(F)n1"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='FC5FNH3', pbc=False)


## Compute the Energy of the Complex

In [43]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:37:35  -113965.758026       13.388060
FIRE:    1 19:37:35  -113971.107330        9.054475
FIRE:    2 19:37:35  -113974.489258        5.663540
FIRE:    3 19:37:35  -113976.622777        3.830898
FIRE:    4 19:37:35  -113978.194831        3.670658
FIRE:    5 19:37:35  -113979.564887        3.055082
FIRE:    6 19:37:35  -113980.764397        3.052824
FIRE:    7 19:37:35  -113981.798261        2.889482
FIRE:    8 19:37:36  -113982.749827        2.645646
FIRE:    9 19:37:36  -113983.636849        2.799871
FIRE:   10 19:37:36  -113984.461243        2.448212
FIRE:   11 19:37:36  -113985.284348        2.972845
FIRE:   12 19:37:36  -113986.081967        2.708278
FIRE:   13 19:37:36  -113986.734390        1.837113
FIRE:   14 19:37:36  -113987.283517        1.814200
FIRE:   15 19:37:36  -113987.856282        2.062816
FIRE:   16 19:37:36  -113988.401614        1.470576
FIRE:   17 19:37:36  -113988.917477        1.875840
FIRE:   18 19:

In [44]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:37:46  -101830.823185       13.485259
FIRE:    1 19:37:46  -101835.448136        8.610290
FIRE:    2 19:37:46  -101838.122465        4.981582
FIRE:    3 19:37:47  -101839.716842        3.382394
FIRE:    4 19:37:47  -101840.931464        3.319123
FIRE:    5 19:37:47  -101841.975349        3.267246
FIRE:    6 19:37:47  -101842.812068        3.110770
FIRE:    7 19:37:47  -101843.529100        2.583975
FIRE:    8 19:37:47  -101844.240364        2.083090
FIRE:    9 19:37:47  -101844.922978        2.500426
FIRE:   10 19:37:47  -101845.593075        3.465517
FIRE:   11 19:37:47  -101846.279106        2.824057
FIRE:   12 19:37:47  -101846.825277        1.630456
FIRE:   13 19:37:47  -101847.251448        2.332886
FIRE:   14 19:37:48  -101847.704508        2.268563
FIRE:   15 19:37:48  -101848.117586        1.781751
FIRE:   16 19:37:48  -101848.545317        2.068580
FIRE:   17 19:37:48  -101848.947926        1.063707
FIRE:   18 19:

In [45]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:37:56   -12158.484558        2.733933
FIRE:    1 19:37:56   -12158.649560        1.616153
FIRE:    2 19:37:56   -12158.762721        1.439182
FIRE:    3 19:37:56   -12158.819426        1.481200
FIRE:    4 19:37:56   -12158.835579        1.115152
FIRE:    5 19:37:56   -12158.854775        0.479626
FIRE:    6 19:37:56   -12158.862552        0.259002
FIRE:    7 19:37:57   -12158.862851        0.250925
FIRE:    8 19:37:57   -12158.863414        0.240539
FIRE:    9 19:37:57   -12158.864177        0.226994
FIRE:   10 19:37:57   -12158.865067        0.212584
FIRE:   11 19:37:57   -12158.866013        0.199901
FIRE:   12 19:37:57   -12158.866967        0.191011
FIRE:   13 19:37:57   -12158.867912        0.186427
FIRE:   14 19:37:57   -12158.868955        0.184537
FIRE:   15 19:37:57   -12158.870128        0.182283
FIRE:   16 19:37:58   -12158.871457        0.174582
FIRE:   17 19:37:58   -12158.872915        0.157572
FIRE:   18 19:

## Compute the energy of the Hydrogen atom

In [46]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:38:00      -31.589716        0.047594
FIRE:    1 19:38:00      -31.589744        0.012353
FIRE:    2 19:38:00      -31.589733        0.031727
FIRE:    3 19:38:00      -31.589737        0.025917
FIRE:    4 19:38:00      -31.589743        0.015339
FIRE:    5 19:38:01      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [47]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: 0.158 eV


## 4-fluoro-2-nitrophenol

In [21]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/4fluoro2nitrophenol_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [22]:
xyz_path = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ni_thd_2_4fluoro2nitrophenol_3_hydride_1_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

In [23]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "Oc1ccc(F)cc1[N+]([O-])=O"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='OC4FC2NO2H4', pbc=False)


In [24]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:20:00  -107572.401979        6.232398
FIRE:    1 04:20:00  -107575.262540        3.128768
FIRE:    2 04:20:00  -107575.246627        6.182753
FIRE:    3 04:20:00  -107575.809060        3.244807
FIRE:    4 04:20:01  -107576.118092        0.965439
FIRE:    5 04:20:01  -107575.994702        2.596631
FIRE:    6 04:20:01  -107576.028208        2.409108
FIRE:    7 04:20:01  -107576.087025        2.039180
FIRE:    8 04:20:01  -107576.156451        1.522686
FIRE:    9 04:20:01  -107576.218710        0.995882
FIRE:   10 04:20:01  -107576.258230        0.742950
FIRE:   11 04:20:01  -107576.269755        0.746830
FIRE:   12 04:20:01  -107576.270840        0.730700
FIRE:   13 04:20:01  -107576.272960        0.716218
FIRE:   14 04:20:01  -107576.276034        0.712657
FIRE:   15 04:20:02  -107576.279938        0.707931
FIRE:   16 04:20:02  -107576.284510        0.702056
FIRE:   17 04:20:02  -107576.289610        0.695062
FIRE:   18 04:

In [25]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:20:09   -90955.226023        6.455554
FIRE:    1 04:20:09   -90957.470190        3.114263
FIRE:    2 04:20:09   -90957.079387        8.551729
FIRE:    3 04:20:09   -90957.846292        3.931984
FIRE:    4 04:20:09   -90958.149829        1.215102
FIRE:    5 04:20:10   -90958.159013        1.205411
FIRE:    6 04:20:10   -90958.175678        1.185965
FIRE:    7 04:20:10   -90958.196934        1.156567
FIRE:    8 04:20:10   -90958.219673        1.116908
FIRE:    9 04:20:10   -90958.241618        1.066682
FIRE:   10 04:20:10   -90958.262226        1.006108
FIRE:   11 04:20:10   -90958.282576        0.936464
FIRE:   12 04:20:10   -90958.306710        0.851373
FIRE:   13 04:20:10   -90958.336219        0.750846
FIRE:   14 04:20:10   -90958.370348        0.636602
FIRE:   15 04:20:11   -90958.405271        0.688176
FIRE:   16 04:20:11   -90958.437690        0.794115
FIRE:   17 04:20:11   -90958.470377        0.876846
FIRE:   18 04:

In [26]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:20:26   -16632.840504        6.366815
FIRE:    1 04:20:26   -16633.595288        3.086174
FIRE:    2 04:20:26   -16633.119242       11.836858
FIRE:    3 04:20:26   -16633.596685        5.414429
FIRE:    4 04:20:26   -16633.772102        2.260839
FIRE:    5 04:20:26   -16633.778710        2.061114
FIRE:    6 04:20:26   -16633.790149        1.688547
FIRE:    7 04:20:27   -16633.803534        1.199369
FIRE:    8 04:20:27   -16633.815967        0.818343
FIRE:    9 04:20:27   -16633.825669        0.826072
FIRE:   10 04:20:27   -16633.832758        0.831850
FIRE:   11 04:20:27   -16633.838945        1.122405
FIRE:   12 04:20:27   -16633.847013        1.265501
FIRE:   13 04:20:27   -16633.858621        1.182122
FIRE:   14 04:20:27   -16633.873631        0.839151
FIRE:   15 04:20:27   -16633.889299        0.697148
FIRE:   16 04:20:27   -16633.902194        0.592269
FIRE:   17 04:20:28   -16633.913341        0.874526
FIRE:   18 04:

In [27]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:20:38      -31.589716        0.047594
FIRE:    1 04:20:39      -31.589744        0.012353
FIRE:    2 04:20:39      -31.589733        0.031727
FIRE:    3 04:20:39      -31.589737        0.025917
FIRE:    4 04:20:39      -31.589743        0.015339
FIRE:    5 04:20:39      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [29]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.492 eV


## 2-mercaptopyridine

In [32]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/aniline_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [33]:
xyz_path = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ni_thd_2_aniline_3_hydride_1_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

In [34]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "Nc1ccccc1"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='NC6H7', pbc=False)


In [35]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:27:42   -72340.432032        1.942990
FIRE:    1 04:27:42   -72340.616893        0.618268
FIRE:    2 04:27:42   -72340.583941        1.413513
FIRE:    3 04:27:42   -72340.646343        1.068818
FIRE:    4 04:27:42   -72340.714534        0.499047
FIRE:    5 04:27:42   -72340.733831        0.620033
FIRE:    6 04:27:43   -72340.736490        0.586447
FIRE:    7 04:27:43   -72340.741414        0.522073
FIRE:    8 04:27:43   -72340.747929        0.489634
FIRE:    9 04:27:43   -72340.755302        0.475058
FIRE:   10 04:27:43   -72340.762970        0.459423
FIRE:   11 04:27:43   -72340.770742        0.443806
FIRE:   12 04:27:43   -72340.778826        0.428887
FIRE:   13 04:27:43   -72340.788645        0.413410
FIRE:   14 04:27:43   -72340.801081        0.427477
FIRE:   15 04:27:43   -72340.816791        0.399609
FIRE:   16 04:27:44   -72340.835670        0.362720
FIRE:   17 04:27:44   -72340.856753        0.345351
FIRE:   18 04:

In [36]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:27:56   -64531.034010        3.356952
FIRE:    1 04:27:57   -64531.305141        2.916706
FIRE:    2 04:27:57   -64531.423913        1.687141
FIRE:    3 04:27:57   -64531.481538        1.599490
FIRE:    4 04:27:57   -64531.549173        1.436583
FIRE:    5 04:27:57   -64531.580839        1.217277
FIRE:    6 04:27:57   -64531.587814        1.142769
FIRE:    7 04:27:57   -64531.594921        1.066069
FIRE:    8 04:27:57   -64531.607619        0.973246
FIRE:    9 04:27:57   -64531.623340        0.966789
FIRE:   10 04:27:57   -64531.639255        0.957150
FIRE:   11 04:27:58   -64531.653221        0.943358
FIRE:   12 04:27:58   -64531.664624        0.924428
FIRE:   13 04:27:58   -64531.674607        0.900139
FIRE:   14 04:27:58   -64531.686542        0.898941
FIRE:   15 04:27:58   -64531.703418        0.905400
FIRE:   16 04:27:58   -64531.727665        0.907124
FIRE:   17 04:27:58   -64531.759357        0.901697
FIRE:   18 04:

In [37]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:28:17    -7825.588005        1.842908
FIRE:    1 04:28:17    -7825.668573        1.131706
FIRE:    2 04:28:17    -7825.705925        0.853407
FIRE:    3 04:28:17    -7825.713616        0.726013
FIRE:    4 04:28:18    -7825.722973        0.497147
FIRE:    5 04:28:18    -7825.727929        0.350825
FIRE:    6 04:28:18    -7825.728839        0.471309
FIRE:    7 04:28:18    -7825.729549        0.443367
FIRE:    8 04:28:18    -7825.730817        0.389457
FIRE:    9 04:28:18    -7825.732379        0.313582
FIRE:   10 04:28:18    -7825.733934        0.253097
FIRE:   11 04:28:18    -7825.735229        0.220218
FIRE:   12 04:28:18    -7825.736155        0.187262
FIRE:   13 04:28:18    -7825.736796        0.195728
FIRE:   14 04:28:19    -7825.737427        0.225791
FIRE:   15 04:28:19    -7825.738303        0.256683
FIRE:   16 04:28:19    -7825.739605        0.238525
FIRE:   17 04:28:19    -7825.741249        0.189644
FIRE:   18 04:

In [38]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:28:30      -31.589716        0.047594
FIRE:    1 04:28:30      -31.589744        0.012353
FIRE:    2 04:28:30      -31.589733        0.031727
FIRE:    3 04:28:30      -31.589737        0.025917
FIRE:    4 04:28:31      -31.589743        0.015339
FIRE:    5 04:28:31      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [39]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: 0.697 eV


# Bidentate Structures

## 2-mercaptopyridine

In [56]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/2mercaptopyridine_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [57]:
xyz_path = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ni_oct_2_2mercaptopyridine_2_hydride_2_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

In [58]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "S=C1NC=CC=C1"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='SCNC4H5', pbc=False)


In [59]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:48:36   -93767.719408        4.768582
FIRE:    1 04:48:36   -93768.915706        2.202763
FIRE:    2 04:48:37   -93769.004613        3.413841
FIRE:    3 04:48:37   -93769.178995        2.488508
FIRE:    4 04:48:37   -93769.371408        1.079424
FIRE:    5 04:48:37   -93769.444411        0.982345
FIRE:    6 04:48:37   -93769.448731        0.945332
FIRE:    7 04:48:37   -93769.456680        0.873597
FIRE:    8 04:48:37   -93769.467066        0.771906
FIRE:    9 04:48:37   -93769.478551        0.647761
FIRE:   10 04:48:37   -93769.490059        0.511775
FIRE:   11 04:48:37   -93769.501115        0.455707
FIRE:   12 04:48:38   -93769.511947        0.544408
FIRE:   13 04:48:38   -93769.524450        0.635854
FIRE:   14 04:48:38   -93769.539729        0.668133
FIRE:   15 04:48:38   -93769.558488        0.620345
FIRE:   16 04:48:38   -93769.580106        0.483414
FIRE:   17 04:48:38   -93769.602491        0.320324
FIRE:   18 04:

In [60]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:48:40   -76223.436265        5.491086
FIRE:    1 04:48:40   -76224.550947        3.115044
FIRE:    2 04:48:40   -76224.671222        3.103353
FIRE:    3 04:48:41   -76224.793823        2.245922
FIRE:    4 04:48:41   -76224.931963        0.929208
FIRE:    5 04:48:41   -76224.986766        0.625547
FIRE:    6 04:48:41   -76224.989107        0.595484
FIRE:    7 04:48:41   -76224.993441        0.592999
FIRE:    8 04:48:41   -76224.999158        0.589300
FIRE:    9 04:48:41   -76225.005588        0.584406
FIRE:   10 04:48:41   -76225.012187        0.578340
FIRE:   11 04:48:41   -76225.018718        0.571147
FIRE:   12 04:48:42   -76225.025310        0.562921
FIRE:   13 04:48:42   -76225.033074        0.552783
FIRE:   14 04:48:42   -76225.042673        0.540478
FIRE:   15 04:48:42   -76225.054595        0.525657
FIRE:   16 04:48:42   -76225.068650        0.507764
FIRE:   17 04:48:42   -76225.083865        0.485924
FIRE:   18 04:

In [61]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:48:44   -17591.029900        3.101191
FIRE:    1 04:48:44   -17591.197803        0.951416
FIRE:    2 04:48:44   -17591.147888        2.909610
FIRE:    3 04:48:44   -17591.202807        1.954049
FIRE:    4 04:48:44   -17591.250253        0.693550
FIRE:    5 04:48:44   -17591.248242        1.056030
FIRE:    6 04:48:45   -17591.250764        0.978003
FIRE:    7 04:48:45   -17591.255172        0.827309
FIRE:    8 04:48:45   -17591.260375        0.614953
FIRE:    9 04:48:45   -17591.265164        0.466943
FIRE:   10 04:48:45   -17591.268612        0.415435
FIRE:   11 04:48:45   -17591.270509        0.369030
FIRE:   12 04:48:45   -17591.271523        0.431640
FIRE:   13 04:48:45   -17591.272828        0.550840
FIRE:   14 04:48:45   -17591.275480        0.587666
FIRE:   15 04:48:45   -17591.279974        0.502208
FIRE:   16 04:48:45   -17591.285555        0.307292
FIRE:   17 04:48:46   -17591.290165        0.234499
FIRE:   18 04:

In [62]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:48:48      -31.589716        0.047594
FIRE:    1 04:48:48      -31.589744        0.012353
FIRE:    2 04:48:48      -31.589733        0.031729
FIRE:    3 04:48:48      -31.589737        0.025916
FIRE:    4 04:48:48      -31.589743        0.015339
FIRE:    5 04:48:48      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [63]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - 2*E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -15.113 eV


## Adsorption energy is too high. Need to debug

## ethanolamine

In [46]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ethanolamine_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [47]:
xyz_path = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ni_oct_2_ethanolamine_2_hydride_2_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

In [48]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "NCCO"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='NC2OH7', pbc=False)


In [49]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:41:13   -58208.186563        6.216393
FIRE:    1 04:41:13   -58209.734419        4.065466
FIRE:    2 04:41:13   -58210.757567        2.766710
FIRE:    3 04:41:13   -58211.373285        1.717239
FIRE:    4 04:41:13   -58211.799997        1.822358
FIRE:    5 04:41:14   -58212.097569        2.247499
FIRE:    6 04:41:14   -58212.466766        2.355983
FIRE:    7 04:41:14   -58212.648920        3.257956
FIRE:    8 04:41:14   -58212.747021        2.183648
FIRE:    9 04:41:14   -58212.854305        1.032544
FIRE:   10 04:41:14   -58212.908660        1.009361
FIRE:   11 04:41:14   -58212.933871        1.535699
FIRE:   12 04:41:14   -58212.981397        1.767795
FIRE:   13 04:41:14   -58213.069212        1.550070
FIRE:   14 04:41:14   -58213.175069        0.933121
FIRE:   15 04:41:15   -58213.264280        0.655393
FIRE:   16 04:41:15   -58213.314168        1.521222
FIRE:   17 04:41:15   -58213.382230        1.562348
FIRE:   18 04:

In [50]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:41:31   -52518.194927        9.949058
FIRE:    1 04:41:31   -52519.930410        4.800738
FIRE:    2 04:41:31   -52520.651799        2.270896
FIRE:    3 04:41:32   -52520.752632        1.797159
FIRE:    4 04:41:32   -52520.882581        1.452762
FIRE:    5 04:41:32   -52520.980721        1.271920
FIRE:    6 04:41:32   -52521.056715        1.497502
FIRE:    7 04:41:32   -52521.155540        1.680829
FIRE:    8 04:41:32   -52521.291041        1.370310
FIRE:    9 04:41:32   -52521.431003        0.855654
FIRE:   10 04:41:32   -52521.544023        0.881260
FIRE:   11 04:41:32   -52521.636354        1.108998
FIRE:   12 04:41:33   -52521.760864        0.892116
FIRE:   13 04:41:33   -52521.886706        0.598839
FIRE:   14 04:41:33   -52521.970508        0.870798
FIRE:   15 04:41:33   -52522.077228        0.767037
FIRE:   16 04:41:33   -52522.166688        0.766414
FIRE:   17 04:41:33   -52522.237630        0.967008
FIRE:   18 04:

In [51]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:41:53    -5724.600817        1.633943
FIRE:    1 04:41:53    -5724.681700        0.580058
FIRE:    2 04:41:53    -5724.660835        1.800217
FIRE:    3 04:41:53    -5724.681093        1.274931
FIRE:    4 04:41:53    -5724.702461        0.591176
FIRE:    5 04:41:53    -5724.707382        0.503223
FIRE:    6 04:41:54    -5724.707824        0.476106
FIRE:    7 04:41:54    -5724.708637        0.423583
FIRE:    8 04:41:54    -5724.709695        0.349477
FIRE:    9 04:41:54    -5724.710846        0.301129
FIRE:   10 04:41:54    -5724.711960        0.295607
FIRE:   11 04:41:54    -5724.712965        0.288835
FIRE:   12 04:41:54    -5724.713877        0.280824
FIRE:   13 04:41:54    -5724.714882        0.270692
FIRE:   14 04:41:54    -5724.716133        0.267923
FIRE:   15 04:41:55    -5724.717769        0.255688
FIRE:   16 04:41:55    -5724.719792        0.225441
FIRE:   17 04:41:55    -5724.721980        0.204351
FIRE:   18 04:

In [52]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 04:42:07      -31.589716        0.047594
FIRE:    1 04:42:08      -31.589744        0.012353
FIRE:    2 04:42:08      -31.589733        0.031727
FIRE:    3 04:42:08      -31.589737        0.025917
FIRE:    4 04:42:08      -31.589743        0.015340
FIRE:    5 04:42:08      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [53]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -14.809 eV


## Wrong Structures

In [43]:
## It has 2 fluorine atoms 
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/pyridine_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [44]:
## It has a hydride attached
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/4methoxyphenol_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')